# Introduction
## Goal. 
The goal of this lab is to investigate two examples of Competitive Co-Evolution: the first one dealing with a robotic prey-predator experiment, the second one dealing with a computational model for sorting algorithms, named Sorting Network.
## Getting started. 
This lab continues the use of the inspyred framework for the Python programming language seen in the previous labs. If you did not participate in the previous labs, you may want to look those over first and then start this lab's exercises. Additionally, in this lab we will use a custom 2-D robotic simulator (for more details, see module 9's exercises), and another Python library for Evolutionary Computation named deap $^{[1]}$. With respect to inspyred, deap has some nice features such as a simple template for co-evolutionary algorithms, and an easy-to-use Genetic Programming implementation. We will see the latter in the next lab.

Note once again that, unless otherwise specified, in this module's exercises we will use real-valued genotypes and that the aim of the algorithms will be to minimize the fitness function f(x), i.e. lower values correspond to a better fitness!

----
[1]: Distributed Evolutionary Algorithms in Python: https://github.com/DEAP/deap

## Utils Code

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# This Code is Heavily Inspired By The YouTuber: Cheesy AI
# Code Changed, Optimized And Commented By: NeuralNine (Florian Dedov)


import math
from typing import Any, Optional

import pygame
from pygame import Surface
import numpy as np
import time

WIDTH = 1920
HEIGHT = 1080

CAR_SIZE_X = 30
CAR_SIZE_Y = 30

BORDER_COLOR = (0, 0, 0, 255)  # Color To Crash on Hit
OBJ_COLOR = (45, 157, 10, 255)
current_generation = 0  # Generation counter
pygame.init()
bck_driver = pygame.display.get_driver()
pygame.quit()


class Car:
    def __init__(
        self,
        pos: tuple[float, float] = (200, 900),
        carSprite: str = "img/car.png",
    ):
        # Load Car Sprite and Rotate
        self.sprite = pygame.image.load(carSprite).convert()  # Convert Speeds Up A Lot
        self.sprite = pygame.transform.scale(self.sprite, (CAR_SIZE_X, CAR_SIZE_Y))
        self.rotated_sprite = self.sprite

        # self.position = [690, 740] # Starting Position
        self.position = list(pos)  # [200, 900]  # Starting Position
        self.angle = 0
        self.speed = 0

        self.speed_set = False  # Flag For Default Speed Later on

        self.center = [
            self.position[0] + CAR_SIZE_X / 2,
            self.position[1] + CAR_SIZE_Y / 2,
        ]  # Calculate Center

        self.radars = []  # List For Sensors / Radars
        self.drawing_radars = []  # Radars To Be Drawn

        self.collision = False  # Boolean To Check If Car is Crashed

        self.distance = 0  # Distance Driven
        self.time = 0  # Time Passed
        self.times = 0.0

    def draw(self, screen: Surface):
        screen.blit(self.rotated_sprite, self.position)  # Draw Sprite
        self.draw_radar(screen)  # OPTIONAL FOR SENSORS

    def draw_radar(self, screen: Surface):
        # Optionally Draw All Sensors / Radars
        for radar in self.radars:
            position = radar[0]
            pygame.draw.line(screen, (0, 255, 0), self.center, position, 1)
            pygame.draw.circle(screen, (0, 255, 0), position, 5)

    def check_collision(self, game_map: Surface):
        self.collision = False
        for point in self.corners:
            # If Any Corner Touches Border Color -> Crash
            # Assumes Rectangle
            if 0 <= point[0] < WIDTH and 0 <= point[1] < HEIGHT:
                if game_map.get_at((int(point[0]), int(point[1]))) == BORDER_COLOR:  # type: ignore
                    self.collision = True

                    break
            else:
                self.collision = True
                break

    def calc_distance(self, p1: list[float], p2: list[float]) -> float:
        return math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

    def check_radar(
        self, degree: float, game_map: Surface, otherCar: Optional[list[int]] = None
    ):
        length = 0
        x = int(
            self.center[0]
            + math.cos(math.radians(360 - (self.angle + degree))) * length
        )
        y = int(
            self.center[1]
            + math.sin(math.radians(360 - (self.angle + degree))) * length
        )
        # print(x)
        # print(y)
        # print(game_map.get_size())
        # While We Don't Hit BORDER_COLOR AND length < 300 (just a max) -> go further and further

        # s
        color = game_map.get_at((x, y))
        cosl = math.cos(0.0174533 * (360 - (self.angle + degree)))
        senl = math.sin(0.0174533 * (360 - (self.angle + degree)))
        # distance_other_car = self.calc_distance([x,y], otherCar) if otherCar is not None else np.inf or distance_other_car <= 15
        while not (color == BORDER_COLOR or color == OBJ_COLOR) and length < 100:  # type: ignore
            length = length + 1
            x = int(self.center[0] + cosl * length)
            y = int(self.center[1] + senl * length)
            # distance_other_car = self.calc_distance([x, y], otherCar) if otherCar is not None else np.inf
            color = game_map.get_at((x, y))

        # Calculate Distance To Border And Append To Radars List
        dist = int(
            math.sqrt(math.pow(x - self.center[0], 2) + math.pow(y - self.center[1], 2))
        )
        self.radars.append([(x, y), dist])

    def _inner_update(self, game_map: Surface, otherCar: Optional[list[int]] = None):
        if not self.speed_set:
            self.speed = 20
            self.speed_set = True

        # Get Rotated Sprite And Move Into The Right X-Direction
        # Don't Let The Car Go Closer Than 20px To The Edge
        self.rotated_sprite = self.rotate_center(self.sprite, self.angle)

        self.position[0] += math.cos(math.radians(360 - self.angle)) * self.speed
        self.position[0] = max(self.position[0], 120)
        self.position[0] = min(self.position[0], WIDTH - 120)

        # Increase Distance and Time
        self.distance += self.speed
        self.time += 1

        # Same For Y-Position
        self.position[1] += math.sin(math.radians(360 - self.angle)) * self.speed
        self.position[1] = max(self.position[1], 90)
        self.position[1] = min(self.position[1], WIDTH - 90)

        # Calculate New Center
        self.center = [
            int(self.position[0]) + CAR_SIZE_X / 2,
            int(self.position[1]) + CAR_SIZE_Y / 2,
        ]

        # Calculate Four Corners
        # Length Is Half The Side
        length = 0.5 * CAR_SIZE_X
        left_top = [
            self.center[0] + math.cos(math.radians(360 - (self.angle + 30))) * length,
            self.center[1] + math.sin(math.radians(360 - (self.angle + 30))) * length,
        ]
        right_top = [
            self.center[0] + math.cos(math.radians(360 - (self.angle + 150))) * length,
            self.center[1] + math.sin(math.radians(360 - (self.angle + 150))) * length,
        ]
        left_bottom = [
            self.center[0] + math.cos(math.radians(360 - (self.angle + 210))) * length,
            self.center[1] + math.sin(math.radians(360 - (self.angle + 210))) * length,
        ]
        right_bottom = [
            self.center[0] + math.cos(math.radians(360 - (self.angle + 330))) * length,
            self.center[1] + math.sin(math.radians(360 - (self.angle + 330))) * length,
        ]
        self.corners = [left_top, right_top, left_bottom, right_bottom]

        # Check Collisions And Clear Radars

        self.check_collision(game_map)
        self.radars.clear()
        # ll = time.time()
        # From -90 To 120 With Step-Size 45 Check Radar
        for d in range(0, 360, 90):
            self.check_radar(d, game_map, otherCar)
        # self.times = time.time() - ll

    def update(self, game_map: Surface, otherCar: Optional[list[int]] = None):
        # Set The Speed To 20 For The First Time
        # Only When Having 4 Output Nodes With Speed Up and Down

        if not self.collision:
            self._inner_update(game_map, otherCar)

        else:  # calculate new position if blocked not update else update
            self.times = 0
            newPosition: list[float] = [0, 0]
            newPosition[0] += int(math.cos(math.radians(360 - self.angle)) * self.speed)
            newPosition[0] = max(self.position[0], 20)
            newPosition[0] = min(self.position[0], WIDTH - 120)
            # Same For Y-Position
            newPosition[1] += int(math.sin(math.radians(360 - self.angle)) * self.speed)
            newPosition[1] = max(self.position[1], 90)
            newPosition[1] = min(self.position[1], HEIGHT - 90)
            x = int(newPosition[0])
            y = int(newPosition[1])

            if not game_map.get_at((x, y)) == BORDER_COLOR:  # type: ignore
                self._inner_update(game_map, otherCar)

        # print(self.times[-1])

    def get_data(self) -> list[float]:
        # Get Distances To Border
        radars = self.radars
        return_values: list[float] = [0, 0, 0, 0]
        for i, radar in enumerate(radars):
            return_values[i] = radar[1] / 100

        return return_values

    def is_collided(self) -> bool:
        # Basic Alive Function
        return self.collision

    def get_reward(self) -> float:
        # Calculate Reward (Maybe Change?)
        # return self.distance / 50.0
        return self.distance / (CAR_SIZE_X / 2)

    def rotate_center(self, image: Surface, angle: float) -> Surface:
        # Rotate The Rectangle
        rectangle = image.get_rect()
        rotated_image = pygame.transform.rotate(image, angle)
        rotated_rectangle = rectangle.copy()
        rotated_rectangle.center = rotated_image.get_rect().center
        rotated_image = rotated_image.subsurface(rotated_rectangle).copy()
        return rotated_image


def run_simulationCoevolution(
    prey: list[Any], predator: list[Any], map: str = "white", render: bool = False
) -> tuple[list[float], list[list[float]], list[list[float]]]:
    # Initialize PyGame And The Display
    # init = time.time()
    if not render:
        os.environ["SDL_VIDEODRIVER"] = "dummy"
    else:
        os.environ["SDL_VIDEODRIVER"] = bck_driver
        if os.environ["SDL_VIDEODRIVER"] == "dummy":
            os.environ.pop("SDL_VIDEODRIVER", None)

    pygame.init()
    screen = None
    if render:
        screen = pygame.display.set_mode(
            (WIDTH, HEIGHT), pygame.FULLSCREEN | pygame.SCALED
        )

    else:
        screen = pygame.display.set_mode((WIDTH, HEIGHT))

    # Clock Settings
    # Font Settings & Loading Map

    clock = pygame.time.Clock()

    clock.tick(10 if render else 100000)
    game_map = pygame.image.load(map).convert()  # Convert Speeds Up A Lot
    # print("Init "+str(time.time()-init))
    global current_generation
    current_generation += 1

    # Simple Counter To Roughly Limit Time (Not Good Practice)
    # counter = 0
    posPredator = [200, 900]
    posPrey = [1730, 165]
    carsPrey = [Car((posPrey[0], posPrey[1])) for _ in range(len(prey))]
    carsPredator = [
        Car((posPredator[0], posPredator[1]), carSprite="img/car_red.png")
        for _ in range(len(prey))
    ]
    # fitness = 0
    obsPrey = [[] for _ in range(len(carsPrey))]
    obsPredator = [[] for _ in range(len(carsPredator))]

    # ss = time.time()
    # print("ffff "+str(len(cars)))
    # ups = []
    timess = [[], [], []]
    for _ in range(250):
        # Exit On Quit Event
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit(0)

        # For Each Car Get The Acton It Takes
        # acts = time.time()
        for i in range(len(prey)):
            # print("cars "+str(i)+"  "+str(cars[i]))
            outtime = time.time()
            obPrey = carsPrey[i].get_data()
            posPrey = [carsPrey[i].center[0] / WIDTH, carsPrey[i].center[1] / HEIGHT]

            obPredator = carsPredator[i].get_data()
            posPredator = [
                carsPredator[i].center[0] / WIDTH,
                carsPredator[i].center[1] / HEIGHT,
            ]

            distance_norm = math.sqrt(
                (
                    (posPrey[0] - posPredator[0]) ** 2
                    + ((posPrey[1] - posPredator[1]) ** 2)
                )
            )

            bearingPredator = (
                np.arctan2(
                    carsPredator[i].center[1] - carsPrey[i].center[1],
                    carsPredator[i].center[0] - carsPrey[i].center[0],
                )
                / np.pi
            )
            bearingPrey = (
                np.arctan2(
                    carsPrey[i].center[1] - carsPredator[i].center[1],
                    carsPrey[i].center[0] - carsPredator[i].center[0],
                )
                / np.pi
            )

            obPrey = [distance_norm, bearingPrey, *obPrey]
            obPredator = [distance_norm, bearingPredator, *obPredator]

            obsPrey[i].append(obPrey)
            obsPredator[i].append(obPredator)

            # s = time.time()
            outputPrey = prey[i].activate(obPrey)

            outputPredator = predator[i].activate(obPredator)

            choicePrey = np.argmax(outputPrey)  # random.randint(0,4)#
            if choicePrey == 0:
                carsPrey[i].angle += 5  # Left
            elif choicePrey == 1:
                carsPrey[i].angle -= 5  # Right
            elif choicePrey == 2:
                carsPrey[i].speed = -10  # Slow Down
            elif choicePrey == 3:
                carsPrey[i].speed = 10  # Speed Up
            else:
                carsPrey[i].speed = 0  # stop

            choicePredator = np.argmax(outputPredator)  # random.randint(0,4)#

            if choicePredator == 0:
                carsPredator[i].angle += 5  # Left
            elif choicePredator == 1:
                carsPredator[i].angle -= 5  # Right
            elif choicePredator == 2:
                carsPredator[i].speed = -10  # Slow Down
            elif choicePredator == 3:
                carsPredator[i].speed = 10  # Speed Up
            else:
                carsPredator[i].speed = 0  # stop
            timess[0].append(time.time() - outtime)
            # Check If Car Is Still Alive
            # Increase Fitness If Yes And Break Loop If Not
            carsPrey[i].update(game_map, carsPredator[i].center)  # type: ignore
            carsPredator[i].update(game_map, carsPrey[i].center)  # type: ignore
            # fitness[i] +=  cars[i].get_reward()

        # print("acts "+str(time.time()-acts))
        # Draw Map And All Cars That Are Alive
        screen.blit(game_map, (0, 0))
        drawtime = time.time()
        for i in range(len(prey)):
            carsPrey[i].draw(screen)
            carsPredator[i].draw(screen)
        timess[2].append(time.time() - drawtime)
        pygame.display.flip()
        clock.tick(24 if render else 100000)
    # print("actions time " + str(sum(timess[0])))
    # print("update time " + str(sum(timess[1])))
    # print("draws time " + str(sum(timess[2])))
    # print("ups "+str(sum(ups))+"   "+str(len(ups)))
    # print("  alla cosa "+str(time.time()-ss))

    pygame.display.quit()
    # print([carsPrey[i].center for i in range(len(prey))])
    # print([carsPredator[i].center for i in range(len(predator))])

    dis = [
        math.sqrt(
            ((carsPrey[i].center[0] - carsPredator[i].center[0]) / WIDTH) ** 2
            + ((carsPrey[i].center[1] - carsPredator[i].center[1]) / HEIGHT) ** 2
        )
        for i in range(len(prey))
    ]
    # print(dis)
    # print("-----------------------------------------------------")
    # print(dis)
    return dis, obsPrey, obsPredator

In [ ]:
class NN:
    def __init__(self, nodes: list[int]):
        self.nodes = nodes
        self.activations: list[list[float]] = [
            [0 for _ in range(node)] for node in nodes
        ]
        self.nweights = sum([
            self.nodes[i] * self.nodes[i + 1] for i in range(len(self.nodes) - 1)
        ])  # nodes[0]*nodes[1]+nodes[1]*nodes[2]+nodes[2]*nodes[3]

        self.weights = [[] for _ in range(len(self.nodes) - 1)]

    def activate(self, inputs: list[float]):
        self.activations[0] = [np.tanh(inputs[i]) for i in range(self.nodes[0])]
        for i in range(1, len(self.nodes)):
            self.activations[i] = [0.0 for _ in range(self.nodes[i])]
            for j in range(self.nodes[i]):
                sum = 0  # self.weights[i - 1][j][0]
                for k in range(self.nodes[i - 1]):
                    sum += self.activations[i - 1][k - 1] * self.weights[i - 1][j][k]
                self.activations[i][j] = np.tanh(sum)
        return np.array(self.activations[-1])

    def set_weights(self, weights: list[float]):
        # self.weights = [[] for _ in range(len(self.nodes) - 1)]
        c = 0
        for i in range(1, len(self.nodes)):
            self.weights[i - 1] = [
                [0 for _ in range(self.nodes[i - 1])] for __ in range(self.nodes[i])
            ]
            for j in range(self.nodes[i]):
                for k in range(self.nodes[i - 1]):
                    self.weights[i - 1][j][k] = weights[c]
                    c += 1
        # print(c)

    def get_list_weights(self) -> list[float]:
        wghts = []
        for i in range(1, len(self.nodes)):
            for j in range(self.nodes[i]):
                for k in range(self.nodes[i - 1]):
                    wghts.append(np.abs(self.weights[i - 1][j][k]))
        return wghts

In [ ]:
from typing import Any

# os.environ["PYGAME_HIDE_SUPPORT_PROMPT"] = "hide"


def eval(
    prey: list[Any],
    predator: list[Any],
    map: str,
    config: dict[str, Any],
    render: bool = False,
):
    sensors = 0 if not config["sensors"] else 4
    nrInputNodes = 2 + sensors  # nrIRSensors + nrDistanceSensor + nrBearingSensor
    nrHiddenNodes = int(config["nrHiddenNodes"])
    nrHiddenLayers = int(config["nrHiddenLayers"])
    nrOutputNodes = 5  # 2

    preyAgents = [
        NN([
            nrInputNodes,
            *[nrHiddenNodes for _ in range(nrHiddenLayers)],
            nrOutputNodes,
        ])
        for _ in range(len(prey))
    ]
    predatorAgents = [
        NN([
            nrInputNodes,
            *[nrHiddenNodes for _ in range(nrHiddenLayers)],
            nrOutputNodes,
        ])
        for _ in range(len(predator))
    ]
    for i in range(len(prey)):
        preyAgents[i].set_weights(prey[i])
    for i in range(len(predator)):
        predatorAgents[i].set_weights(predator[i])

    dis, obsPrey, obsPredator = run_simulationCoevolution(
        preyAgents, predatorAgents, map=map, render=render
    )

    return dis, obsPrey, obsPredator

In [ ]:
from random import shuffle
from typing import Any, Callable, Optional
from inspyred.ec import (
    variators,
    observers,
    terminators,
    replacers,
    selectors,
    Bounder,
    EvolutionaryComputation,
)
from numpy.typing import NDArray
from random import Random


class ArchiveSolutions:
    def __init__(self, size: Optional[int] = None):
        self.candidates = []
        self.fitnesses = []
        self.size = size

    def appendToArchive(
        self,
        candidate: NDArray[np.float64],
        fitness: Optional[float] = None,
        maximize: Optional[bool] = None,
        archiveType: Optional[str] = None,
    ):
        if (
            self.size is None
            or archiveType is None
            or archiveType == "HALLOFFAME"
            or (len(self.candidates) < self.size and len(self.fitnesses) < self.size)
        ):
            if candidate in self.candidates:
                index = self.candidates.index(candidate)
                self.fitnesses[index] = fitness
            else:
                self.candidates.append(candidate)
                self.fitnesses.append(fitness)
        else:
            if archiveType == "GENERATION":
                # delete the oldest candidate and add the new one
                if candidate in self.candidates:
                    index = self.candidates.index(candidate)
                    self.fitnesses[index] = fitness
                else:
                    del self.candidates[0]
                    del self.fitnesses[0]
                    self.candidates.append(candidate)
                    self.fitnesses.append(fitness)
            elif archiveType == "BEST":
                # find worst candidate in the archive
                if maximize:
                    worstFitness = min(self.fitnesses)
                else:
                    worstFitness = max(self.fitnesses)
                worstIndex = self.fitnesses.index(worstFitness)
                #  replace it if the new candidate is better than the worst candidate in the archive
                if (fitness > worstFitness and maximize) or (
                    fitness < worstFitness and not maximize
                ):
                    if candidate in self.candidates:
                        index = self.candidates.index(candidate)
                        self.fitnesses[index] = fitness
                    else:
                        del self.candidates[worstIndex]
                        del self.fitnesses[worstIndex]
                        self.candidates.append(candidate)
                        self.fitnesses.append(fitness)

    def getIndexesOfOpponents(self, numOpponents: int) -> list[int]:
        archiveSize = len(self.candidates)
        indexes = list(range(archiveSize))
        shuffle(indexes)
        numOpponents = min(numOpponents, archiveSize)
        return indexes[0:numOpponents]


# --------------------------------------------------------------------------- #
# Utility functions


def getAggregateFitness(
    fitness_tmp: NDArray[np.float64], maximize: bool, archiveUpdate: str
) -> float:
    if archiveUpdate == "AVERAGE":
        fitness = np.mean(fitness_tmp)
    elif archiveUpdate == "WORST":
        if maximize:
            fitness = np.min(fitness_tmp)
        else:
            fitness = np.max(fitness_tmp)

    return fitness.item()  # type: ignore


def getIndexOfBest(fitnesses: list[float], maximize: bool) -> int:
    if maximize:
        bestFitness = max(fitnesses)
    else:
        bestFitness = min(fitnesses)
    bestIndex = fitnesses.index(bestFitness)
    return bestIndex


# --------------------------------------------------------------------------- #
#  The robot evaluator class


class RobotEvaluator:
    def __init__(
        self,
        config: dict[str, Any],
        eval_func_prey: Callable[[float, float, float, float, int], float],
        eval_func_predator: Callable[[float, float, float, float, int], float],
        name: str,
        q_mine: Any,
        q_his: Any,
        seed: int,
        maximize: bool,
    ):
        self.config = config
        self.name = name
        self.q_mine = q_mine
        self.q_his = q_his
        self.seed = seed
        self.shared_variables = config["shared_variables"]
        sensors = 0 if not config["sensors"] else 4
        nrInputNodes = 2 + sensors  # nrIRSensors + nrDistanceSensor + nrBearingSensor
        nrHiddenNodes = int(config["nrHiddenNodes"])
        nrHiddenLayers = int(config["nrHiddenLayers"])
        nrOutputNodes = 5  # 2

        # calculate the no. of weights
        fka = NN([
            nrInputNodes,
            *[nrHiddenNodes for _ in range(nrHiddenLayers)],
            nrOutputNodes,
        ])

        nrWeights = fka.nweights

        self.geneMin = -3.0  # float(parameters["geneMin"])
        self.geneMax = 3.0  # float(parameters["geneMax"])
        self.nrTimeStepsGen = 0  # int(parameters["nrTimeStepsGen"])
        self.fitness_evaluator_prey = eval_func_prey
        self.fitness_evaluator_predator = eval_func_predator

        self.nrWeights = nrWeights
        self.seed = seed
        self.bounder = Bounder(
            [self.geneMin] * self.nrWeights, [self.geneMax] * self.nrWeights
        )
        self.maximize = maximize

        self.genCount = 0
        self.archiveType = config["archiveType"]
        self.numOpponents = config["numOpponents"]
        self.updateBothArchives = config["updateBothArchives"]
        self.showArchives = config["showArchives"]
        self.numGen = config["numGen"]
        self.archiveUpdate = config["archiveUpdate"]

    def generator(self, random: Random, args: dict[str, Any]) -> list[float]:
        return [
            random.uniform(self.geneMin, self.geneMax) for _ in range(self.nrWeights)
        ]

    def _avg_distance(self, obs: list[list[float]]) -> float:
        tmp = []
        for i in range(len(obs)):
            tmp.append(obs[i][0])

        return np.mean(tmp).item()

    def _best_distance(self, obs: list[list[float]], maximize: bool) -> float:
        distance = [obs[i][0] for i in range(len(obs))]
        if maximize:
            return max(distance)
        else:
            return min(distance)

    def _first_contact(self, obs: list[list[float]]) -> int:
        t = 250
        distance = [obs[i][0] for i in range(len(obs))]
        for i in range(len(distance)):
            if distance[i] <= 15.0:
                return i
        return t

    def evaluator(
        self, candidates: list[list[float]], args: dict[str, Any]
    ) -> list[float]:
        # get lock
        self.q_mine.get()

        #  identify candidates and opponents
        preys: Any = None
        predators: Any = None
        if self.genCount == 0:
            # at the first generation, let all preys compete against all predators
            preys = self.shared_variables.initialPreys.candidates
            predators = self.shared_variables.initialPredators.candidates
        else:
            #  at the next generations, let all preys (predators) compete against individuals in the archives of predators (preys)
            if self.name == "Preys":
                preys = candidates
                if self.archiveType == "HALLOFFAME":
                    indexesOfOpponents = (
                        self.shared_variables.archivePredators.getIndexesOfOpponents(
                            self.numOpponents
                        )
                    )
                    predators = []
                    archiveSize = len(self.shared_variables.archivePredators.candidates)
                    for i in range(min(self.numOpponents, archiveSize)):
                        predators.append(
                            self.shared_variables.archivePredators.candidates[
                                indexesOfOpponents[i]
                            ]
                        )
                else:
                    predators = self.shared_variables.archivePredators.candidates
            elif self.name == "Predators":
                predators = candidates
                if self.archiveType == "HALLOFFAME":
                    indexesOfOpponents = (
                        self.shared_variables.archivePreys.getIndexesOfOpponents(
                            self.numOpponents
                        )
                    )
                    preys = []
                    archiveSize = len(self.shared_variables.archivePreys.candidates)
                    for i in range(min(self.numOpponents, archiveSize)):
                        preys.append(
                            self.shared_variables.archivePreys.candidates[
                                indexesOfOpponents[i]
                            ]
                        )
                else:
                    preys = self.shared_variables.archivePreys.candidates

        #  create the candidate populations to evaluate
        # we assume that the population is split in two halves, one for preys and one for predators
        """
            n (preys) repeated m (predators) times
            vs
            1 predator -repeated n (preys)- repeated m (predators) times

            [                   [               -
            prey_1              predator_1      |
            prey_2              predator_1      |
            ...                 ...             n
            prey_n              predator_1      |
            ]                   ]               -

            [                   [
            prey_1              predator_2
            prey_2              predator_2
            ...                 ...
            prey_n              predator_2
            ]                   ]

            ...                 ...

            [                   [
            prey_1              predator_m
            prey_2              predator_m
            ...                 ...
            prey_n              predator_m
            ]                   ]
        """
        repeatedPreys = []
        repreatedPredator = []
        # append preys
        for prey in preys:
            for predator in predators:
                repeatedPreys.append(prey)
                repreatedPredator.append(predator)

        # run the simulator
        dis, obsP, _ = eval(
            repeatedPreys,
            repreatedPredator,
            "img/" + self.config["map"],
            self.config,
            False,
        )
        numRobots = len(dis)
        # TODO: calculate fitness here
        fitnessTmpPrey = []
        fitnessTmpPredator = []

        for i in np.arange(numRobots):
            finalDistanceToTarget = dis[i]  #
            avgDistance = self._avg_distance(obsP[i])
            minDistanceToTarget = self._best_distance(obsP[i], False)
            maxDistanceToTarget = self._best_distance(obsP[i], True)
            timeToContact = self._first_contact(obsP[i])
            fitnessPrey = self.fitness_evaluator_prey(
                finalDistanceToTarget,
                avgDistance,
                minDistanceToTarget,
                maxDistanceToTarget,
                timeToContact,
            )
            fitnessPredator = self.fitness_evaluator_predator(
                finalDistanceToTarget,
                avgDistance,
                minDistanceToTarget,
                maxDistanceToTarget,
                timeToContact,
            )

            fitnessTmpPrey.append(fitnessPrey)
            fitnessTmpPredator.append(fitnessPredator)

            """
            if i < numRobots/2:
                # preys
                fitnessTmp.append(minDistanceToTarget)
            else:
                # predators
                fitnessTmp.append(timeToContact)
            """

        # update fitness and archives
        fitness_preys = []
        fitness_predators = []

        numPredators = len(predators)
        numPreys = len(preys)
        # print(fitnessTmp)
        # --------------------------------------------------------------------------- #
        if self.updateBothArchives:
            # (update alternative) in this case at each step we update both archives
            # update fitness of preys
            for i in range(numPreys):
                prey = preys[i]
                indexes = np.arange(i * numPredators, (i + 1) * numPredators)
                fitness_prey = getAggregateFitness(
                    np.array(fitnessTmpPrey)[indexes],
                    self.shared_variables.problemPreysMaximize,
                    self.archiveUpdate,
                )
                fitness_preys.append(fitness_prey)
            if self.archiveType == "GENERATION" or self.archiveType == "HALLOFFAME":
                # get best prey in the current population
                indexOfBestPrey = getIndexOfBest(
                    fitness_preys, self.shared_variables.problemPreysMaximize
                )
                bestPrey = preys[indexOfBestPrey]
                bestPreyFitness = fitness_preys[indexOfBestPrey]
                # update archive of preys
                self.shared_variables.archivePreys.appendToArchive(
                    bestPrey,
                    bestPreyFitness,
                    self.shared_variables.problemPreysMaximize,
                    self.archiveType,
                )
            elif self.archiveType == "BEST":
                # update archive of preys
                for i in range(numPreys):
                    prey = preys[i]
                    fitness_prey = fitness_preys[i]
                    self.shared_variables.archivePreys.appendToArchive(
                        prey,
                        fitness_prey,
                        self.shared_variables.problemPreysMaximize,
                        self.archiveType,
                    )

            # update fitness of predators
            for i in range(numPredators):
                predator = predators[i]
                indexes = np.arange(i, numPreys * numPredators, numPredators)
                fitness_predator = getAggregateFitness(
                    np.array(fitnessTmpPredator)[indexes],
                    self.shared_variables.problemPredatorsMaximize,
                    self.archiveUpdate,
                )
                fitness_predators.append(fitness_predator)
            if self.archiveType == "GENERATION" or self.archiveType == "HALLOFFAME":
                # get best predator in the current population
                indexOfBestPredator = getIndexOfBest(
                    fitness_predators, self.shared_variables.problemPredatorsMaximize
                )
                bestPredator = predators[indexOfBestPredator]
                bestPredatorFitness = fitness_predators[indexOfBestPredator]
                # update archive of predators
                self.shared_variables.archivePredators.appendToArchive(
                    bestPredator,
                    bestPredatorFitness,
                    self.shared_variables.problemPredatorsMaximize,
                    self.archiveType,
                )
            elif self.archiveType == "BEST":
                # update archive of predators
                for i in range(numPredators):
                    predator = predators[i]
                    fitness_predator = fitness_predators[i]
                    self.shared_variables.archivePredators.appendToArchive(
                        predator,
                        fitness_predator,
                        self.shared_variables.problemPredatorsMaximize,
                        self.archiveType,
                    )
                    ############################# here
        else:
            # (update alternative) in this case at each step we update only one archive
            if self.name == "Preys":
                # update fitness of preys
                for i in range(numPreys):
                    prey = preys[i]
                    indexes = np.arange(i * numPredators, (i + 1) * numPredators)
                    fitness_prey = getAggregateFitness(
                        np.array(fitnessTmpPrey)[indexes],
                        self.shared_variables.problemPreysMaximize,
                        self.archiveUpdate,
                    )
                    fitness_preys.append(fitness_prey)
                if self.archiveType == "GENERATION" or self.archiveType == "HALLOFFAME":
                    # get best prey in the current population
                    indexOfBestPrey = getIndexOfBest(
                        fitness_preys, self.shared_variables.problemPreysMaximize
                    )
                    bestPrey = preys[indexOfBestPrey]
                    bestPreyFitness = fitness_preys[indexOfBestPrey]
                    # update archive of preys
                    self.shared_variables.archivePreys.appendToArchive(
                        bestPrey,
                        bestPreyFitness,
                        self.shared_variables.problemPreysMaximize,
                        self.archiveType,
                    )
                elif self.archiveType == "BEST":
                    # update archive of preys
                    for i in range(numPreys):
                        prey = preys[i]
                        fitness_prey = fitness_preys[i]
                        self.shared_variables.archivePreys.appendToArchive(
                            prey,
                            fitness_prey,
                            self.shared_variables.problemPreysMaximize,
                            self.archiveType,
                        )
            elif self.name == "Predators":
                # update fitness of predators
                for i in range(numPredators):
                    predator = predators[i]
                    indexes = np.arange(i, numPreys * numPredators, numPredators)
                    fitness_predator = getAggregateFitness(
                        np.array(fitnessTmpPredator)[indexes],
                        self.shared_variables.problemPredatorsMaximize,
                        self.archiveUpdate,
                    )
                    fitness_predators.append(fitness_predator)
                if self.archiveType == "GENERATION" or self.archiveType == "HALLOFFAME":
                    # get best predator in the current population
                    indexOfBestPredator = getIndexOfBest(
                        fitness_predators,
                        self.shared_variables.problemPredatorsMaximize,
                    )
                    bestPredator = predators[indexOfBestPredator]
                    bestPredatorFitness = fitness_predators[indexOfBestPredator]
                    # update archive of predators
                    self.shared_variables.archivePredators.appendToArchive(
                        bestPredator,
                        bestPredatorFitness,
                        self.shared_variables.problemPredatorsMaximize,
                        self.archiveType,
                    )
                elif self.archiveType == "BEST":
                    # update archive of predators
                    for i in range(numPredators):
                        predator = predators[i]
                        fitness_predator = fitness_predators[i]
                        self.shared_variables.archivePredators.appendToArchive(
                            predator,
                            fitness_predator,
                            self.shared_variables.problemPredatorsMaximize,
                            self.archiveType,
                        )

        # --------------------------------------------------------------------------- #
        fitness = []
        if self.name == "Preys":
            fitness = fitness_preys
        elif self.name == "Predators":
            fitness = fitness_predators

        # show archives
        if self.showArchives:
            archive = "Archive preys: [ "
            for x in self.shared_variables.archivePreys.fitnesses:
                archive += "{:.4f}".format(x) + " "
            print(archive + "]")

            archive = "Archive predators: [ "
            for x in self.shared_variables.archivePredators.fitnesses:
                archive += "{:.4f}".format(x) + " "
            print(archive + "]")

        print(self.name, self.genCount, "/", self.numGen)

        # increment generation counter
        self.genCount += 1

        # release lock
        self.q_his.put(1)

        return fitness


# --------------------------------------------------------------------------- #


def runEA(
    problem: Any, display: bool, rng: Random, config: dict[str, Any]
) -> list[list[float]]:
    # --------------------------------------------------------------------------- #
    # EA configuration

    # the evolutionary algorithm (EvolutionaryComputation is a fully configurable evolutionary algorithm)
    #  standard GA, ES, SA, DE, EDA, PAES, NSGA2, PSO and ACO are also available
    shared_variable = config["shared_variables"]
    print(shared_variable)
    ea = EvolutionaryComputation(rng)

    # observers: provide various logging features
    if display:
        ea.observer = [observers.file_observer]  # type: ignore
        # observers.stats_observer
        # observers.file_observer,
        # observers.best_observer,
        # observers.population_observer,

    #  selection operator
    # ea.selector = selectors.truncation_selection
    # ea.selector = selectors.uniform_selection
    # ea.selector = selectors.fitness_proportionate_selection
    # ea.selector = selectors.rank_selection
    ea.selector = selectors.tournament_selection

    # variation operators (mutation/crossover)
    ea.variator = [  # type: ignore
        variators.gaussian_mutation,
        variators.n_point_crossover,
    ]
    # variators.random_reset_mutation,
    # variators.inversion_mutation,
    # variators.uniform_crossover,
    # variators.partially_matched_crossover,

    # replacement operator
    # ea.replacer = replacers.truncation_replacement
    # ea.replacer = replacers.steady_state_replacement
    # ea.replacer = replacers.random_replacement
    # ea.replacer = replacers.plus_replacement
    # ea.replacer = replacers.comma_replacement
    # ea.replacer = replacers.crowding_replacement
    # ea.replacer = replacers.simulated_annealing_replacement
    # ea.replacer = replacers.nsga_replacement
    # ea.replacer = replacers.paes_replacement
    ea.replacer = replacers.generational_replacement

    # termination condition
    # ea.terminator = terminators.evaluation_termination
    # ea.terminator = terminators.no_improvement_termination
    # ea.terminator = terminators.diversity_termination
    # ea.terminator = terminators.time_termination
    ea.terminator = terminators.generation_termination

    # --------------------------------------------------------------------------- #

    initialPopulation = None
    if problem.name == "Preys":
        initialPopulation = shared_variable.initialPreys.candidates
    elif problem.name == "Predators":
        initialPopulation = shared_variable.initialPredators.candidates

    # run the EA
    final_pop = ea.evolve(
        seeds=initialPopulation,
        generator=problem.generator,
        evaluator=problem.evaluator,
        bounder=problem.bounder,
        maximize=problem.maximize,
        pop_size=config["popSize"],
        max_generations=config["numGen"],
        # max_evaluations=config['numEval'],
        tournament_size=config["tournamentSize"],
        mutation_rate=config["mutationRate"],
        gaussian_mean=config["gaussianMean"],
        gaussian_stdev=config["gaussianStdev"],
        crossover_rate=config["crossoverRate"],
        num_crossover_points=config["numCrossoverPoints"],
        num_selected=config["selectionSize"],
        num_elites=config["numElites"],
        statistics_file=open("results/stats_" + problem.name + ".csv", "w"),
        individuals_file=open("results/individuals_" + problem.name + ".csv", "w"),
    )

    # --------------------------------------------------------------------------- #

    return final_pop

In [ ]:
import pickle


def post_eval(
    file_path: str,
) -> tuple[list[float], list[list[float]], list[list[float]]]:
    inputs = pickle.load(open(file_path, "rb"))
    best = inputs[0]
    config = inputs[1]
    dis, obs_prey, obs_predator = eval(
        [best[0]], [best[1]], map="utils/utils_10/white.png", config=config, render=True
    )
    print(dis)
    return dis, obs_prey, obs_predator


class cfgs:
    archivePreys = None
    archivePredators = None
    initialPreys = None
    initialPredators = None
    problemPreysMaximize = None  # e.g. maximize (final/min) distance from predator or maximize time-to-contact
    problemPredatorsMaximize = (
        None  # e.g. minimize (final/min) distance from prey or minimize time-to-contact
    )

# Exercise 1
In this exercise we will replicate the prey-predator competitive co-evolution experiments we have seen during the lecture. To do so, we will use a custom 2-D simulator. In this case by default the simulator generates an "empty" (without obstacles) arena $1920 \times 1080$ px, with two cars that at the beginning of the simulation are placed at opposite corners of the arena. One robot has the role of a "prey" (blue), the other acts as a "predator" (red). The goal of the prey is to avoid being "captured" (i.e., get in contact with) the predator, whereas the goal of the predator is to "capture" (get in touch with) the prey. 

By default, both cars are controlled by a Feed Forward Neural Network (FFNN) with 4 lidar sensor inputs to detect walls, 2 inputs for distance and bearing towards the "target" -in this case towards the other robot-, 5 hidden nodes, and 5 output nodes for controlling the behaviour of the robot (for more details, see module 9's exercises). All nodes use a tanh activation function, with inputs/outputs normalized in $[0, 1]$ and weights evolved in the range $[−3, 3]$.

In order to co-evolve the cars behaviors, two separate populations of preys and predators are evolved synchronously by two Evolutionary Algorithms that run in separate alternating threads (i.e., the evaluation part of the algorithm alternates between the two threads: evaluate preys, evaluate predators, evaluate preys, ...). Both algorithms can be parametrized differently (but, by default they use the same parameters), and are configured to keep an archive of best solutions (i.e., FFNN controllers) found during the evolution.

At each generation, each algorithm simulates its own solutions in the current population against (a subset of) the solutions taken from the archive kept by the other algorithm. For efficiency purposes, the simulation takes as input a list of $numRobots$ candidate solutions that is split in two halves, the first one containing preys, the second one containing predators. Then the simulator lets each $i$-th robot (a prey), for $i$ in $[0,numRobots/2)$, "compete" against the $(i+numRobots/2)$-th robot (the corresponding predator). Therefore each prey and each predator can be repeated in the list multiple times, in order to generate all the needed pairwise competitions between preys and predators. Before starting the experiments, spend some time to have look at the script `utils/utils_10/robot_coevolution.py` and understand its main steps (in particular, see the method evaluator of the class `RobotEvaluator`).

Depending on how fitness is defined for both preys and predators (you can change it in the next cell), different robot behaviors can be obtained. For each robot, the simulator returns three main quantities that can be used/combined differently to drive the co-evolution in different ways, namely:

1. finalDistanceToTarget: the final (measured at the end of the simulation) distance to the "target" robot.
2. minDistanceToTarget: the minimum (measured during the simulation) distance to the "target" robot.
3. timeToContact: the time to contact (in timesteps, in the range [0,nrTimeStepsGen]).
    
By default, the preys are evolved to maximize their minDistanceToTarget, while the predators are evolved to minimize it. Please note that the two distance metrics range in $[0, \sqrt{2}]$ (in px) where $\sqrt{2}$ is pre-computed as the maximum distance normalized in the given environment (length of the diagonal of the arena). Also, note that, due to the aforementioned structure of the list of candidate solutions taken as input, preys are kept in the first $numRobots/2$ elements of the list, while predators are kept in the remaining elements, such that different fitness functions can be used for preys and predators.

Furthermore, the way the two algorithms update the corresponding archives of best solutions can be controlled by the following parameters:

- numOpponents: the number of opponents against which each robot competes at each generation (default: 1).
- archiveType: the way competition with individuals from the archive is performed; possible values are {GENERATION, HALLOFFAME, BEST} (default: BEST). When GENERATION is selected, generational competition is performed, i.e. each algorithm keeps an archive containing one best solution for each of the previous numOpponents generations, such that at each generation each robot competes against the best opponents from those numOpponents generations (Master Tournaments). Similarly, when HALLOFFAME is selected each algorithm keeps an archive containing one best solution for each of the previous generations, however in this case there is no limit on the number of solutions kept in the archive (whose size increases along generations), and numOpponents indicates the number of solutions which are randomly sampled from the archive to perform pairwise competitions (see the lecture slides). When BEST is selected, a greedy approach is taken: in thisase indeed each algorithm keeps in the archive the best numOpponents opponents from all previous generations (not necessarily one per generation), and each robot competes against those opponents.
- archiveUpdate: the way fitness is aggregated for each robot; possible values are {WORST, AVERAGE} (default: WORST). When WORST is selected, each robot competes against numOpponents opponents from the other algorithm's archive and its final fitness is set to its worst value obtained across numOpponents competitions (worst-case scenario). When AVERAGE is selected, the final fitness of each robot is set to its average value obtained across numOpponents competitions.
- updateBothArchives: the way archives are updated at each generation; possible values are {True, False} (default: False). When False is selected, each algorithm updates only its own archive. When True is selected, each algorithm also recomputes the fitness of the opponents and updates the other algorithm's archive if needed (this is a non-standard feature).
    
Consider the following experiments:
- Try out different parameter combinations of numOpponents, archiveType, archiveUpdate, and updateBothArchives, and observe what kind of robot behavior is evolved. Can you find cases where the prey "wins"? Can you find cases where the predator "wins"?
- Try to change the fitness formulation and observe what kind of behavior is evolved. Remember to change the two flags problemPreysMaximize and problemPredatorsMaximize properly, according to the way you defined the fitness function.
- (Optional) Try to change the EA's and FFNN's parameters to see if/how results change depending on those values.

In [ ]:
from queue import Queue
import threading

import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import numpy as np

from utils.inspyred_utils import NumpyRandomWrapper

config = {
    "sensors": True,
    "nrHiddenNodes": 5,
    "nrHiddenLayers": 1,
    "map": "white.png",  # parameters for standard GA
    "popSize": 10,  # population size
    "numGen": 10,  # used with generation_termination
    "tournamentSize": 2,  # tournament size (default 2)
    "mutationRate": 0.2,  # mutation rate, per gene (default 0.1)
    "gaussianMean": 0,  # mean of the Gaussian distribution used for mutation
    "gaussianStdev": 0.1,  # std. dev. of the Gaussian distribution used for mutation
    "crossoverRate": 1.0,  # rate at which crossover is performed (default 1.0)
    "numCrossoverPoints": 1,  # number of crossover points used (default 1)
    "numElites": 1,  # no. of elites (i.e. best individuals that are kept in the population # parameters for competitive coevolution
    "selectionSize": 10,  # selection size (i.e. how many individuals are selected for reproduction)
    "numOpponents": 5,  # number of opponents against which each robot competes at each generation
    "archiveType": "GENERATION",  # possible types: {GENERATION,HALLOFFAME,BEST}
    "archiveUpdate": "Worst",  # possible types: {WORST,AVERAGE}
    "updateBothArchives": False,  # True is each generation should update both archives, False otherwise
    "display": True,
    "showArchives": False,
}

config["selectionSize"] = config["popSize"]

# 1. Generational competition: the archive is filled with the best individuals from previous n generations (e.g. n=5)
# 2. Hall-of-Fame: each new individual is tested against *all best opponents* obtained so far.
#    NOTE: Using this method, the no. of tournaments increases along generations!
#    However, it is sufficient to test new individuals only against a limited sample of n opponents (e.g. n=10)
# 3. Best competition: the archive is filled with the best n (e.g. n=5) individuals from *all* previous generations


def fitness_eval_prey(
    finalDistanceToTarget: float,
    avgDistanceToTarget: float,
    minDistanceToTarget: float,
    maxDistanceToTarget: float,
    timeToContact: float,
):
    print("final distance to target: ", finalDistanceToTarget)
    print("avg distance to target: ", avgDistanceToTarget)
    print("min distance to target: ", minDistanceToTarget)
    print("max distance to target: ", maxDistanceToTarget)
    print("time to contact: ", timeToContact)
    fitness = finalDistanceToTarget
    return fitness


def fitness_eval_predator(
    finalDistanceToTarget: float,
    avgDistanceToTarget: float,
    minDistanceToTarget: float,
    maxDistanceToTarget: float,
    timeToContact: float,
):
    print("final distance to target: ", finalDistanceToTarget)
    print("avg distance to target: ", avgDistanceToTarget)
    print("min distance to target: ", minDistanceToTarget)
    print("max distance to target: ", maxDistanceToTarget)
    print("time to contact: ", timeToContact)
    fitness = finalDistanceToTarget
    return fitness


cc = cfgs()

# These two archives keep the best preys and best predators

if config["archiveType"] == "GENERATION" or config["archiveType"] == "BEST":
    cc.archivePreys = ArchiveSolutions(config["numOpponents"])  # type: ignore
    cc.archivePredators = ArchiveSolutions(config["numOpponents"])  # type: ignore
elif config["archiveType"] == "HALLOFFAME":
    cc.archivePreys = ArchiveSolutions()  # type: ignore
    cc.archivePredators = ArchiveSolutions()  # type: ignore

#  the initial popolations (we need to make initialize them externally to initialize the archives)
cc.initialPreys = ArchiveSolutions(config["popSize"])  # type: ignore
cc.initialPredators = ArchiveSolutions(config["popSize"])  # type: ignore

# TODO: change maximize flag depending on how fitness is defined
cc.problemPreysMaximize = True
cc.problemPredatorsMaximize = False

config["shared_variables"] = cc  # type: ignore

# --------------------------------------------------------------------------- #

seed = 41
rng = NumpyRandomWrapper(seed)
#  the following queues allow the two threads to alternate their execution
qAB = Queue()
qBA = Queue()

# create the robot evaluator instances
problemPreys = RobotEvaluator(
    config,
    fitness_eval_prey,
    fitness_eval_predator,
    "Preys",
    qAB,
    qBA,
    seed,
    cc.problemPreysMaximize,  # type: ignore
)
problemPredators = RobotEvaluator(
    config,
    fitness_eval_prey,
    fitness_eval_predator,
    "Predators",
    qBA,
    qAB,
    seed,
    cc.problemPredatorsMaximize,  # type: ignore
)

# create the initial populations
for i in np.arange(config["popSize"]):  # type: ignore
    candidatePrey = [
        (problemPreys.geneMax - problemPreys.geneMin) * rng.random_sample()
        + problemPreys.geneMin
        for _ in range(problemPreys.nrWeights)
    ]
    cc.initialPreys.appendToArchive(candidatePrey)  # type: ignore
for i in np.arange(config["popSize"]):  # type: ignore
    candidatePredator = [
        (problemPredators.geneMax - problemPredators.geneMin) * rng.random_sample()
        + problemPredators.geneMin
        for _ in range(problemPredators.nrWeights)
    ]
    cc.initialPredators.appendToArchive(candidatePredator)  # type: ignore

t1 = threading.Thread(target=runEA, args=(problemPreys, config["display"], rng, config))
t2 = threading.Thread(
    target=runEA, args=(problemPredators, config["display"], rng, config)
)

# this is needed to unlock the thread "Preys" first
qAB.put(1)

t1.start()
t2.start()

t1.join()
t2.join()

if config["display"]:
    """
    # rerun every prey in the archive against every predator in the archive
    preysPredators = []

    # append preys
    for predator in archivePredators.candidates:
        for prey in archivePreys.candidates:
            preysPredators.append(prey)
    # append predators
    for predator in archivePredators.candidates:
        for prey in archivePreys.candidates:
            preysPredators.append(predator)
    """

    # rerun the best prey in the archive against the best predator in the archive
    preysPredators = []
    indexOfBestPrey = getIndexOfBest(cc.archivePreys.fitnesses, cc.problemPreysMaximize)  # type: ignore
    bestPrey = cc.archivePreys.candidates[indexOfBestPrey]  # type: ignore
    bestPreyFitness = cc.archivePreys.fitnesses[indexOfBestPrey]  # type: ignore
    indexOfBestPredator = getIndexOfBest(  # type: ignore
        cc.archivePredators.fitnesses,  # type: ignore
        cc.problemPredatorsMaximize,  # type: ignore
    )
    bestPredator = cc.archivePredators.candidates[indexOfBestPredator]  # type: ignore
    bestPredatorFitness = cc.archivePredators.fitnesses[indexOfBestPredator]  # type: ignore
    print("prey " + str(bestPreyFitness))
    print("predator " + str(bestPredatorFitness))
    preysPredators.append(bestPrey)
    preysPredators.append(bestPredator)

    statsPreys = np.transpose(
        np.loadtxt(open("results/stats_Preys.csv", "r"), delimiter=",")
    )
    statsPredators = np.transpose(
        np.loadtxt(open("results/stats_Predators.csv", "r"), delimiter=",")
    )

    # plot fitness trends of preys and predators
    ax: list[Axes]
    f, ax = plt.subplots(1, 2, figsize=(11, 5))  # type: ignore
    ax[0].plot(statsPreys[2], label="Worst")
    ax[0].plot(statsPreys[3], label="Best")
    ax[0].plot(statsPreys[4], label="Median")
    ax[0].plot(statsPreys[5], label="Mean")
    ax[0].set_xlabel("Generation")
    ax[0].set_ylabel("Fitness")
    ax[0].set_title("Preys")
    ax[0].legend()

    ax[1].plot(statsPredators[2], label="Worst")
    ax[1].plot(statsPredators[3], label="Best")
    ax[1].plot(statsPredators[4], label="Median")
    ax[1].plot(statsPredators[5], label="Mean")
    ax[1].set_xlabel("Generation")
    ax[1].set_ylabel("Fitness")
    ax[1].set_title("Predators")
    ax[1].legend()

    plt.show()
    with open("results/bests.pkl", "wb") as file:
        pickle.dump([[bestPrey], [bestPredator], f"img/{config['map']}", config], file)

In [ ]:
post_eval("results/bests.pkl")

# Exercise 2
In this exercise we will use a competitive co-evolutionary approach for evolving a Sorting Network (SN)$^{[1]}$. A SN is an abstract mathematical model of a network of wires and comparator modules (connectors) that is used to sort a sequence of numbers fed as input to the network. Each comparator connects two wires and sorts the values by outputting the smaller value to one wire, and the larger value to the other. For further details on the theory behind SNs, refer to the  corresponding Wikipedia page. For illustration purposes, an example of SN is shown in the figure.

![sn.png](img/sn.png)

The code of this exercise is based on one of the deap examples and provides a nice template for a generic competitive co-evolutionary algorithm in this framework. Similarly to the previous exercise, also in this case two separate populations are kept, one for hosts (where each host is a candidate sorting network) and one for parasites (where each parasite is an array of a given number of shuffled sequences to be sorted). For simplicity, we consider here a binary sorting problem, where input sequences are (unsorted) binary strings of fixed size, such as $\{1,0,0,0,1,1,0,1\}$. The size of the evolved SNs (also called depth, i.e. its number of connectors) is instead variable and evolves with the hosts$^{[2]}$. The goal of a host is to sort all sequences in the competing parasite, while the goal of a parasite is to induce errors in the competing host. The fitness of a host is therefore calculated as the total number of sequences that it could not sort properly, from those
sequences present in the parasite against which that host competed. The fitness of the parasite is exactly the same value. Obviously, the fitness of hosts (number of sorting errors) must be minimized, while the fitness of parasites should be maximized.

While the deap library is based on some different concepts and implementation details with respect to the inspyred library we have used so far, its working principles are quite straightforward and can be understood rather easily. Take some time to have a look at the source code in the cell below and in the script `exercise_sortingnetwork.py` (note however that the implementation of the Sorting Network is available in the module `sortingnetwork.py` in the `utils/deapCoev folder`). The relevant parameters of the Evolutionary Algorithm, hosts and parasites, can be found at the beginning of the script. In particular, consider the parameters `INPUTS, POP_SIZE_HOSTS, POP_SIZE_PARASITES, HOF_SIZE, MAXGEN, H_CXPB, H_MUTPB, P_CXPB, P_MUTPB, H_TRNMT_SIZE, P_TRNMT_SIZE, P_NUM_SEQ`. Note that in this case (differently from the previous exercise) the two populations are evolved within a single thread, and that at each generation all hosts in the current population are tested against all parasites in the current population. A Hall-of-Fame is kept to store the SNs displaying the best performance across generations, and updated whenever a new SN has a better performance (smaller number of sorting errors) than the worst SN in the Hall-of-Fame. The final output of the script is a graphical representation of the best SN in the Hall-of-Fame, and the number of
sorting errors it suffers on all possible input sequences of fixed input size equal to `INPUTS`. Also, the usual plot with the min/max/avg fitness trends is provided.

Run the cell below to perform the competitive co-evolutionary experiment. Also in this case you can pass as argument to the script a specific seed.

 - Is the co-evolutionary algorithm able to evolve an optimal (without sorting errors) SN, in the default configuration?
 - Try to investigate this problem in different configurations. In particular, focus on the effect of the size of the input sequences (`INPUTS`), the number of input sequences per parasite (`P_NUM_SEQ`), and the two population sizes (`POP_SIZE_HOSTS` and` POP_SIZE_PARASITES`). If needed, also change the size of the Hall-of-Fame (`HOF_SIZE`) and the number of generations  (`MAXGEN`). What conclusions can you draw? For instance: What makes the problem harder? What is the effect of `P_NUM_SEQ`? What can you do to solve the harder problem instances?
 
---

[3]: Link to https://en.wikipedia.org/wiki/Sorting_network

[4]: Note that the depth of a SN is a measure of its algorithmic complexity. Optimal (minimum-depth) SNs are currently known only up to a inputs sequences of size equal to 17, see the corresponding Wikipedia page. In principle, one could use EAs also for minimizing the depth, together with the sorting errors, either based on a single objective (with calarization) or on a multi-objective approach optimizing depth and sorting errors separately.

In [ ]:
# -*- coding: utf-8 -*-

#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.


from deap import base
from deap import creator
from deap import tools


from deapCoev.sortingnetwork import (
    genNetwork,
    evalNetwork,
    mutNetwork,
    cloneHost,
    getParasite,
    mutParasite,
    main,
    cloneParasite,
)


config = {
    "INPUTS": 5,  # length of the input sequence to sort
    "POP_SIZE_HOSTS": 300,  # population size for hsots
    "POP_SIZE_PARASITES": 300,  # population size for parasites
    "HOF_SIZE": 1,  # size of the Hall-of-Fame
    "MAXGEN": 50,  # number of generations
    "H_CXPB": 0.5,  # crossover probability for hosts
    "H_MUTPB": 0.3,  # mutation probability for hosts
    "P_CXPB": 0.5,  # crossover probability for parasites
    "P_MUTPB": 0.3,  # mutation probability for parasites
    "H_TRNMT_SIZE": 3,  # tournament size for hosts
    "P_TRNMT_SIZE": 3,  # tournament size for parasites
    "P_NUM_SEQ": 20,  # number of shuffled sequences for each parasite
}

"""
-------------------------------------------------------------------------
"""

# --------------------------------------------------------------------
# The EA parametrization

# this four lines simply tells DEAP that
# 1. hosts want to minimize (the sorting errors)
# 2. parasites want to maximize (the sorting errors)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Host", list, fitness=creator.FitnessMin)  # type: ignore
creator.create("Parasite", list, fitness=creator.FitnessMax)  # type: ignore

htoolbox = base.Toolbox()
ptoolbox = base.Toolbox()

# register the initialization operators for hosts
htoolbox.register(
    "network",
    genNetwork,
    dimension=config["INPUTS"],
    min_size=config["INPUTS"],
    max_size=config["INPUTS"] * 2,
)
htoolbox.register("individual", tools.initIterate, creator.Host, htoolbox.network)  # type: ignore
htoolbox.register("population", tools.initRepeat, list, htoolbox.individual)  # type: ignore

# register the initialization operators for parasites
ptoolbox.register("parasite", getParasite, dimension=config["INPUTS"])
# NOTE: each parasite is actually an array of P_NUM_SEQ shuffled sequences (not just one)
ptoolbox.register(
    "individual",
    tools.initRepeat,
    creator.Parasite,  # type: ignore
    ptoolbox.parasite,  # type: ignore
    config["P_NUM_SEQ"],
)
ptoolbox.register("population", tools.initRepeat, list, ptoolbox.individual)  # type: ignore

# register the evaluation/crossover/mutation/selection/clone operators for hosts
# we keep the additional specific parameters as they are
htoolbox.register("evaluate", evalNetwork, dimension=config["INPUTS"])
htoolbox.register("mate", tools.cxTwoPoint)
htoolbox.register(
    "mutate",
    mutNetwork,
    dimension=config["INPUTS"],
    mutpb=0.2,
    addpb=0.01,
    delpb=0.01,
    indpb=0.05,
)
htoolbox.register("select", tools.selTournament, tournsize=config["H_TRNMT_SIZE"])
htoolbox.register("clone", cloneHost)

# register the crossover/mutation/selection/clone operators for parasites
# note that in this case an evaluation function is not defined explicitly
# (parasite"s fitness is the same as the corresponding host, see below)
# we keep the additional specific parameters as they are
ptoolbox.register("mate", tools.cxTwoPoint)
ptoolbox.register("indMutate", tools.mutFlipBit, indpb=0.05)
ptoolbox.register("mutate", mutParasite, indmut=ptoolbox.indMutate, indpb=0.05)  # type: ignore
ptoolbox.register("select", tools.selTournament, tournsize=config["P_TRNMT_SIZE"])
ptoolbox.register("clone", cloneParasite)

# --------------------------------------------------------------------
seed = 0
main(seed, creator, htoolbox, ptoolbox, config);